Получаем данные с coinmarketcap.com

In [1]:
import requests
import json

c_url = "https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit=1500&sortBy=market_cap&sortType=desc&convert=USD,BTC,ETH&cryptoType=all&tagType=all&audited=false&aux=ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,max_supply,circulating_supply,total_supply,volume_7d,volume_30d,self_reported_circulating_supply,self_reported_market_cap"
c_resp = requests.get(c_url)
c_resp.raise_for_status() # для проверки ошибок
c_data = c_resp.json()["data"]["cryptoCurrencyList"]

Получаем данные с simpleswap

In [2]:
s_url = "https://simpleswap.io/api/v3/currencies?fixed=false&includeDisabled=false"
s_resp = requests.get(s_url)
s_resp.raise_for_status()
s_data = s_resp.json()

Собираем список монет simpleswap для сравнения с coinmarketcap

In [3]:
s_symbols = {coin['symbol'].upper() for coin in s_data}

Находим монеты из coinmarketcap, которых нет в simpleswap (volume24h - берем в usd)

In [5]:
missing_coins = []
for coin in c_data:
    symbol = coin["symbol"].upper()
    if symbol not in s_symbols:
        missing_coins.append({
            "name": coin["name"],
            "symbol": symbol,
            "volume24h": coin["quotes"][2]["volume24h"]
        })

Сортируем и смотрим на список

In [6]:
missing_coins_sorted = sorted(missing_coins, key=lambda x: x["volume24h"], reverse=True)

In [7]:
for i in missing_coins_sorted[0:3]:
    print(i)

{'name': 'First Digital USD', 'symbol': 'FDUSD', 'volume24h': 7857897576.342774}
{'name': 'BNB', 'symbol': 'BNB', 'volume24h': 4486726378.835662}
{'name': 'Kite', 'symbol': 'KITE', 'volume24h': 919128081.5591974}


Теперь переместим список монет в базу данных, где выполнял первое задание

In [8]:
import psycopg2

conn = psycopg2.connect(dbname="evercode_lab", user="postgres", password="Aa110604", host="localhost", port="5432")
cursor = conn.cursor()

cursor.execute("""
create table if not exists missing_coin (
    id serial primary key,
    name text,
    symbol text,
    volume24h numeric
)
""")
conn.commit()
# Очистим таблицу, чтоб не дублировать данные
cursor.execute("truncate table missing_coin restart identity") # restart identity - сбрасывает связанные sequence
conn.commit()

# Вставляем сами данные
for coin in missing_coins_sorted:
    cursor.execute(
        "insert into missing_coin (name, symbol, volume24h) values(%s, %s, %s)", # с плейсхолдерами
        (coin["name"], coin["symbol"], coin["volume24h"])
    )

conn.commit()

Проверим данные, выполнив запрос к бд и закроем соединение

Всего нашлось монет

In [9]:
cursor.execute("select count(*) from missing_coin")
cnt = cursor.fetchone()[0]
print(cnt)

532


In [10]:
cursor.execute("select * from missing_coin limit 10")
rows = cursor.fetchall()
for row in rows:
    print(row)

cursor.close()
conn.close()

(1, 'First Digital USD', 'FDUSD', Decimal('7857897576.342774'))
(2, 'BNB', 'BNB', Decimal('4486726378.835662'))
(3, 'Kite', 'KITE', Decimal('919128081.5591974'))
(4, '48 Club Token', 'KOGE', Decimal('622254544.0667877'))
(5, 'Horizen', 'ZEN', Decimal('347729888.5516523'))
(6, 'Marina Protocol', 'BAY', Decimal('342700083.68909454'))
(7, 'Quq', 'QUQ', Decimal('289920745.0114728'))
(8, 'APRO', 'AT', Decimal('268055020.74886018'))
(9, 'MetaArena', 'TIMI', Decimal('253657464.79248327'))
(10, 'Momentum', 'MMT', Decimal('226191763.6728252'))
